### Change Input Shape of Model

In [17]:
import keras 
def change_model (model , new_input_shape = (None,None,None,3)):
    '''1st method'''
    '''model.layers.pop(0)
    newInputLayer = keras.models.Input(batch_shape= new_input_shape)
    newOutputs = model(newInputLayer)
    newModel = keras.models.Model(newInputLayer, newOutputs)
    newModel.summary()
    '''
    
    
    
    '''2nd method'''
    model_config = model.get_config()
    input_layer_name = model_config['layers'][0]['name']
    model_config['layers'][0] = {
                      'name': 'new_input',
                      'class_name': 'InputLayer',
                      'config': {
                          'batch_input_shape':new_input_shape,
                          'dtype': 'float32',
                          'sparse': False,
                          'name': 'new_input'
                      },
                      'inbound_nodes': []
                  }
    model_config['layers'][1]['inbound_nodes'] = [[['new_input', 0, 0, {}]]]
    model_config['input_layers'] = [['new_input', 0, 0]]
    
    new_model = model.__class__.from_config(model_config,custom_objects={})
    

    #rebuild model architecture by exporting and importing via json
    #new_model = keras.models.model_from_json(newModel.to_json())

    #copy weights from old model to the new one
    for layer in new_model.layers:
        try:
            layer.set_weights(model.get_layer(name=layer.name).get_weights())
            print("Loaded layer {} with weights of previously loaded model".format(layer.name))
        except:
            print("Could not transfer weight for layer {}".format(layer.name))
    
    return new_model

'''
oldPnet = keras.models.load_model("Pnet_full_model.h5")
newPnet = change_model(oldPnet , (1,500,500,3))
newPnet.summary()
'''


'\noldPnet = keras.models.load_model("Pnet_full_model.h5")\nnewPnet = change_model(oldPnet , (1,500,500,3))\nnewPnet.summary()\n'

# Make the Pnet model

In [14]:
from keras.layers import Conv2D, Input,MaxPool2D, Reshape,Activation,Flatten, Dense, Permute
from keras.models import Model, Sequential
from keras.layers.advanced_activations import PReLU
import numpy as np
import cv2
import tools_matrix as tools

def create_Pnet(weight, height , weight_path = 'model12old.h5'):
    input = Input(shape=[weight, height, 3])
    x = Conv2D(10, (3, 3), strides=1, padding='valid', name='conv1')(input)
    x = PReLU(shared_axes=[1,2],name='PReLU1')(x)
    x = MaxPool2D(pool_size=2)(x)
    x = Conv2D(16, (3, 3), strides=1, padding='valid', name='conv2')(x)
    x = PReLU(shared_axes=[1,2],name='PReLU2')(x)
    x = Conv2D(32, (3, 3), strides=1, padding='valid', name='conv3')(x)
    x = PReLU(shared_axes=[1,2],name='PReLU3')(x)
    classifier = Conv2D(2, (1, 1), activation='softmax', name='conv4-1')(x)
    #bbox_regress = Conv2D(4, (1, 1), name='conv4-2')(x)
    model = Model([input], [classifier])#,bbox_regress])
    model.load_weights(weight_path, by_name=True)
    return model

w = 27
h = 27
Pnet = create_Pnet(w,h,r'12net.h5')
Pnet.save("Pnet_full_model.h5")

### Pnet Summary

In [10]:
import keras

model = keras.models.load_model("Pnet_full_model.h5")
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 35, 35, 3)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 33, 33, 10)        280       
_________________________________________________________________
PReLU1 (PReLU)               (None, 33, 33, 10)        10        
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 16, 16, 10)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 14, 14, 16)        1456      
_________________________________________________________________
PReLU2 (PReLU)               (None, 14, 14, 16)        16        
_________________________________________________________________
conv3 (Conv2D)               (None, 12, 12, 32)        4640      
__________

D:\Conda Installation\envs\gpu_python3.6\lib\site-packages\keras\models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


# Make the Rnet Model

In [10]:
from keras.layers import Conv2D, Input,MaxPool2D, Reshape,Activation,Flatten, Dense, Permute
from keras.models import Model, Sequential
from keras.layers.advanced_activations import PReLU
import numpy as np
import cv2
import tools_matrix as tools
def create_Rnet (weight_path = 'model24.h5'):
    input = Input(shape=[24, 24, 3])  # change this shape to [None,None,3] to enable arbitraty shape input
    x = Conv2D(28, (3, 3), strides=1, padding='valid', name='conv1')(input)
    x = PReLU(shared_axes=[1, 2], name='prelu1')(x)
    x = MaxPool2D(pool_size=3,strides=2, padding='same')(x)

    x = Conv2D(48, (3, 3), strides=1, padding='valid', name='conv2')(x)
    x = PReLU(shared_axes=[1, 2], name='prelu2')(x)
    x = MaxPool2D(pool_size=3, strides=2)(x)

    x = Conv2D(64, (2, 2), strides=1, padding='valid', name='conv3')(x)
    x = PReLU(shared_axes=[1, 2], name='prelu3')(x)
    x = Permute((3, 2, 1))(x)
    x = Flatten()(x)
    x = Dense(128, name='conv4')(x)
    x = PReLU( name='prelu4')(x)
    classifier = Dense(2, activation='softmax', name='conv5-1')(x)
    bbox_regress = Dense(4, name='conv5-2')(x)
    model = Model([input], [classifier, bbox_regress])
    model.load_weights(weight_path, by_name=True)
    return model


Rnet = create_Rnet(r'24net.h5')
Rnet.save("Rnet_full_model.h5")

### Rnet Summary

In [11]:
import keras
model = keras.models.load_model("Rnet_full_model.h5")
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 24, 24, 3)    0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 22, 22, 28)   784         input_10[0][0]                   
__________________________________________________________________________________________________
prelu1 (PReLU)                  (None, 22, 22, 28)   28          conv1[0][0]                      
__________________________________________________________________________________________________
max_pooling2d_20 (MaxPooling2D) (None, 11, 11, 28)   0           prelu1[0][0]                     
__________________________________________________________________________________________________
conv2 (Con

D:\Conda Installation\envs\gpu_python3.6\lib\site-packages\keras\models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


# Make the Onet Model

In [2]:
from keras.layers import Conv2D, Input,MaxPool2D, Reshape,Activation,Flatten, Dense, Permute
from keras.models import Model, Sequential
from keras.layers.advanced_activations import PReLU
import numpy as np
import cv2
import tools_matrix as tools
def create_Onet( weight_path = 'model48.h5'):
    input = Input(shape = [48,48,3])
    x = Conv2D(32, (3, 3), strides=1, padding='valid', name='conv1')(input)
    x = PReLU(shared_axes=[1,2],name='prelu1')(x)
    x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    x = Conv2D(64, (3, 3), strides=1, padding='valid', name='conv2')(x)
    x = PReLU(shared_axes=[1,2],name='prelu2')(x)
    x = MaxPool2D(pool_size=3, strides=2)(x)
    x = Conv2D(64, (3, 3), strides=1, padding='valid', name='conv3')(x)
    x = PReLU(shared_axes=[1,2],name='prelu3')(x)
    x = MaxPool2D(pool_size=2)(x)
    x = Conv2D(128, (2, 2), strides=1, padding='valid', name='conv4')(x)
    x = PReLU(shared_axes=[1,2],name='prelu4')(x)
    x = Permute((3,2,1))(x)
    x = Flatten()(x)
    x = Dense(256, name='conv5') (x)
    x = PReLU(name='prelu5')(x)

    #classifier = Dense(2, activation='softmax',name='conv6-1')(x)
    bbox_regress = Dense(4,name='conv6-2')(x)
    #landmark_regress = Dense(10,name='conv6-3')(x)
    model = Model([input], [bbox_regress])#classifier, bbox_regress, landmark_regress])#, landmark_regress])
    model.load_weights(weight_path, by_name=True)

    return model


Rnet = create_Onet(r'48net.h5')
Rnet.save("Onet_full_model.h5")

### Onet Summary

In [53]:
import keras
model = keras.models.load_model("Onet_full_model.h5")
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_40 (InputLayer)        (None, 48, 48, 3)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 46, 46, 32)        896       
_________________________________________________________________
prelu1 (PReLU)               (None, 46, 46, 32)        32        
_________________________________________________________________
max_pooling2d_68 (MaxPooling (None, 23, 23, 32)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 21, 21, 64)        18496     
_________________________________________________________________
prelu2 (PReLU)               (None, 21, 21, 64)        64        
_________________________________________________________________
max_pooling2d_69 (MaxPooling (None, 10, 10, 64)        0         
__________